In [2]:
import json
import time
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity, ttl, persistence_path):
        self.cache = OrderedDict()
        self.capacity = capacity
        self.ttl = ttl
        self.persistence_path = persistence_path

    def get(self, key):
        if key in self.cache and not self.is_expired(key):
            self.cache.move_to_end(key)
            return self.cache[key]['data']
        return None

    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = {'data': value, 'timestamp': time.time()}
        if len(self.cache) > self.capacity:
            oldest = next(iter(self.cache))
            del self.cache[oldest]

    def is_expired(self, key):
        return (time.time() - self.cache[key]['timestamp']) > self.ttl

    def persist(self):
        # Convert to a format that is JSON serializable
        with open(self.persistence_path, 'w') as f:
            json.dump({k: {'data': str(v['data']), 'timestamp': v['timestamp']} for k, v in self.cache.items()}, f)

    def restore(self):
        try:
            with open(self.persistence_path, 'r') as f:
                loaded_cache = json.load(f)
                self.cache = OrderedDict((k, {'data': v['data'], 'timestamp': v['timestamp']}) for k, v in loaded_cache.items())
        except FileNotFoundError:
            print("Cache file not found, starting with an empty cache.")
    
    def print_cache(self):
        # This method prints the current state of the cache
        print("Cache contents:")
        for key, value in self.cache.items():
            data = value['data']
            timestamp = value['timestamp']
            current_time = time.time()
            age = current_time - timestamp
            print(f"Key: {key}, Data: {data}, Timestamp: {timestamp}, Age: {age:.2f} seconds")

cache = LRUCache(capacity=100, ttl=3600, persistence_path='cache.json')
cache.restore()

Cache file not found, starting with an empty cache.
